<a href="https://colab.research.google.com/github/geetua/fastai-course22/blob/master/Ch4_learner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import os
os.makedirs ('/content/gdrive/My Drive', exist_ok = True)
import fastbook
#fastbook.setup_book()

Create your own implementation of Learner from scratch, based on the training loop shown in this chapter.

In [3]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [4]:
path = untar_data(URLs.MNIST_SAMPLE)

1

In [5]:
#hide
Path.BASE_PATH = path

In [67]:
# all files in valid 3+7
len(path.ls()[0].ls()[0].ls()) + len(path.ls()[0].ls()[1].ls()) #2038

# all files in train 3+7
len(path.ls()[2].ls()[0].ls()) + len(path.ls()[2].ls()[1].ls()) #12396



12396

In [7]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png')...]

In [8]:
# lets create the 3s and 7s data set
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

In [9]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_threes.shape

torch.Size([6131, 28, 28])

# Creating a data block


In [77]:
dblock = DataBlock(
            blocks=(ImageBlock, CategoryBlock),
            get_items=get_image_files,
            splitter=GrandparentSplitter(train_name='train', valid_name='valid'),
            get_y=parent_label)
dls = dblock.dataloaders(path)

In [78]:
dls

In [41]:
items = get_image_files(path)
items

(#14434) [Path('valid/3/6801.png'),Path('valid/3/7104.png'),Path('valid/3/5814.png'),Path('valid/3/6381.png'),Path('valid/3/6481.png'),Path('valid/3/2533.png'),Path('valid/3/369.png'),Path('valid/3/8070.png'),Path('valid/3/7841.png'),Path('valid/3/7690.png')...]

In [15]:
im = PILImageBW.create(items[0])
im

In [73]:
gs_splits = GrandparentSplitter(train_name='train', valid_name='valid')

In [76]:
splits = gs_splits(items)
type(splits)

tuple

In [75]:
splits[0][:5], splits[1][:5]
len(splits[0]), len(splits[1])

(12396, 2038)

# Creating the Model


In [79]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

# Defining Optimizer

In [80]:
opt_func = partial(Adam, lr=3e-3)

# Loss **Function**

In [82]:

def mnist_loss(predictions, targets):
    return torch.where(targets==1, 1-predictions, predictions).mean()

loss_func=mnist_loss

# Metrics Definition

In [83]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()
metrics=batch_accuracy

# Defining Learner

In [86]:
def Learner(dls, model, loss_func, opt_func, metrics=None, cbs=None):
        return LearnerBase(dls, model, loss_func, opt_func, metrics, cbs)

class LearnerBase:
  def __init__(self, dls, model, loss_func, opt_func, metrics=None, cbs=None):
                      self.dls = dls
                      self.model = model
                      self.loss_func = loss_func
                      self.opt_func = opt_func
                      self.metrics = metrics
                      self.cbs = cbs